In [ ]:
import pandas as pd
import datetime as dt
import footballData as fd
import numpy as np
import random
import matplotlib.pyplot as plt
import datetime
import time
import Elo as ELO
import seaborn as sns
import math

df1888=pd.read_csv('englandthesis.csv')
df1888

startdate=1958
fulldf=df1888[df1888['Season']>=startdate]
fulldf=fulldf.sort_values('Date')
fulldf=fulldf.reset_index()
fulldf=fulldf.replace('Milton Keynes Dons', 'Wimbledon')
fulldf

def CalendarYear(Date):
    return int(Date.split('-')[0])
CalendarYear(fulldf['Date'][0])
def Month(Date):
    return int(Date.split('-')[1])
fulldf['CalendarYear']=fulldf.apply(lambda row: CalendarYear(row['Date']), axis=1)
fulldf['Month']=fulldf.apply(lambda row: Month(row['Date']), axis=1)
fulldf

import datetime
from datetime import datetime
def DT(Date):
    return datetime.strptime(Date, '%Y-%m-%d')
fulldf['Date']=fulldf.apply(lambda row: DT(row['Date']), axis=1)

def to_integer(current_time):
    return (current_time-fulldf['Date'][0]).days
fulldf['Day#']=fulldf.apply(lambda row: to_integer(row['Date']), axis=1)
fulldf

In [ ]:
import copy
from copy import deepcopy
def g(sigma2, q):
    return 1/np.sqrt(1+3*q**2*sigma2/(np.pi**2))

def E(r, rj, g, Home, HomeAway):
    return 1/(1+10**(-g*(r-rj+HomeAway*Home)/400))

def Res(HomeG, AwayG):
    if(HomeG>AwayG):
        return 1
    elif(AwayG>HomeG):
        return 0
    else:
        return .5
def insideD(g, E):
    return E*(1-E)*(g**2)

def d2(q, dlist):
    if dlist!=0:
        return 1/(q**2*dlist)
    else:
        return 10**10

def ratingupdate(rating, q, d, RD, sums):
    return rating+(q/(1/RD**2+1/d))*sums      
def rdupdate(rd, d):
    return np.sqrt(1/(1/rd**2+1/d))

def timeupdate(c, rd, maxrd):
    return min(np.sqrt(rd**2+c**2), maxrd)

def lik(E, R):
    if(R==1):
        return np.log(E)
    elif (R==.5):
        return np.log(np.sqrt(E*(1-E)))
    elif (R==0):
        return np.log(1-E)
    


In [ ]:
initdictionary={}
initvalues=[1500,1500,1500,1500]
initialrows=[]
Rd=350


#This needs to be rewritten so we don't have to run it over an dover again
for i in range(len(fulldf['home'].unique())):
    Team=fulldf['home'].unique()[i]
    Teamdf=fulldf[fulldf['home']==Team].reset_index()
    Tier=int(Teamdf['tier'][0])
    LastMatch=fulldf['Day#'][0]
    dcalc=0
    scalc=0
    initialrow=[Team,initvalues[Tier-1], Rd,Tier,  dcalc,scalc ]
    initialrows.append(initialrow)
#Needs to be rewritten to optimize using BuildEloDict Function
for a in range(len(initialrows)):
    initdictionary[initialrows[a][0]]=initialrows[a][1:]
import time
def Glickologlik(params):
    dicts=[]
    rows2=[]
    ratingperiodlength=7.0
    c=params['c']
    h=80
    hk=params['hk']
    #minrd=params['minrd']
    maxrd=params['maxrd']
    curtime=time.time()
    rows=len(fulldf)
    q=np.log(10)/400
    logliks=0.0
    hburn=10
    burn=25
    startyear=fulldf['Season'].values[0]
    year=copy.deepcopy(startyear)
    startday=fulldf['Season']
    Glickodictionary=copy.deepcopy(initdictionary)
    for key in Glickodictionary:
        Glickodictionary[key][1]=maxrd
    currentyear=fulldf['Season'].values[0]
    startdate=fulldf['Date'][0]
    GameCounter=0.
    Rs=0.
    Es=0.
    totaltime=fulldf['Day#'][rows-1]-fulldf['Day#'][0]
    day=0.
    #print(Glickodictionary)
    while day<=totaltime:
        for key in Glickodictionary:
            Glickodictionary[key][1]=timeupdate(c, Glickodictionary[key][1], maxrd)
        ratingperioddf=fulldf[(fulldf['Day#'] >=day) & (fulldf['Day#']<day+ratingperiodlength)].reset_index()
        if len(ratingperioddf!=0):
            Rs=0.
            Es=0.

            for ind, row in ratingperioddf.iterrows():
                if(row['Season']!=year):
                    newdf2=fulldf[fulldf['Season']==row['Season']]
                    for key in Glickodictionary:
                        teamdf=newdf2[newdf2['home']==key].reset_index()
                        if(len(teamdf)!=0):
                            firstgame=teamdf.iloc[0]
                            Glickodictionary[key][2]=firstgame['tier']
                    dicts.append([row['Season'],h, copy.deepcopy(Glickodictionary)])
                #print('sup')
                Home=row['home']
                Away=row['visitor']
                HomeGoals=row['hgoal']
                AwayGoals=row['vgoal']
                year=row['Season']
                date=row['Date']
                tier=row['tier']
                Homestats=Glickodictionary[Home]
                Awaystats=Glickodictionary[Away]
                HomeRating=Homestats[0]
                AwayRating=Awaystats[0]
                HomeRd=Homestats[1]
                AwayRd=Awaystats[1]
                HomeG=g(AwayRd**2, q)
                AwayG=g(HomeRd**2, q)
                pG=g(HomeRd**2+AwayRd**2, q)
                HomeE=E(HomeRating, AwayRating, HomeG, h, 1)
                AwayE=E(AwayRating, HomeRating, AwayG, h, -1)
                pE=E(HomeRating, AwayRating, pG, h, 1)
                HRes=Res(HomeGoals, AwayGoals)
                ARes=Res(AwayGoals, HomeGoals)

                Glickodictionary[Home][4]+=HomeG*(HRes-HomeE)
                Glickodictionary[Away][4]+=AwayG*(ARes-AwayE)
                Glickodictionary[Home][3]+=insideD(HomeG, HomeE)
                Glickodictionary[Away][3]+=insideD(AwayG, AwayE)
                Rs+=HRes
                Es+=pE
                row=[Home, Away,date, year,tier, HomeRating, AwayRating, h, HomeRd, AwayRd, pG, HomeGoals, AwayGoals, HRes, pE]
                rows2.append(row)
                if(year>startyear+burn):
                    logliks+=lik(pE, HRes)
                    

            if(year>startyear+hburn):
                h+=hk*(Rs-Es)

            for key in Glickodictionary:
                Glickod=d2(q, Glickodictionary[key][3])
                Glickodictionary[key][0]=ratingupdate(Glickodictionary[key][0],q, Glickod, Glickodictionary[key][1], Glickodictionary[key][4])
                Glickodictionary[key][1]=rdupdate(Glickodictionary[key][1], Glickod)
                Glickodictionary[key][3]=0
                Glickodictionary[key][4]=0
        day+=ratingperiodlength
    print(params ,logliks,Glickodictionary['Tottenham Hotspur'][0], time.time()-curtime)
    #print(Glickodictionary)
    return dicts

In [ ]:
params={'c': 6.611411615490053, 'hk': 0.26568045135393087, 'maxrd': 114.83368210171477} 

poissonrows=Glickologlikboost(best2)

In [ ]:
initdictionary

In [ ]:
YearGlickos=Glickologlikboost(best2)

In [ ]:
YearGlickos[56]

In [ ]:
YearGlickos

In [ ]:
def SeasonSimulator1st(year):
    maxrd=77.77234622658803
    c=0
    ratingperiodlength=7.0
    rc=0.79591
    hc=1.12149  
    intc=0.26566 
    hk=0.2752882185619584*10/46
    q=np.log(10)/400
    for i in range(len(YearGlickos)):
        if YearGlickos[i][0]==year:
            yeardict=YearGlickos[i][2]
            h=YearGlickos[i][1]
    yeardf=fulldf[fulldf['Season']==year].reset_index()
    del yeardf['index']
    premdf=yeardf[yeardf['tier']==1].reset_index()
    del premdf['index']
    del premdf['level_0']
    del premdf['Unnamed: 0']
    teams=premdf['home'].unique()
    teamdict={}
    for team in teams:
        yeardictent=yeardict[team]
        teamdict[team]=[yeardict[team][0], yeardict[team][1], 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    
    #Index 0: rating, index 1: rd, index 2: dcalc, index 3: scalc, index 4: wins, index5: draws, index6: losses, index7: points, index8: matches.
    #index 9: GF, index10: GA, index11: GD
    currentday=min(premdf['Day#'])
    lastday=max(premdf['Day#'])
    #print(teamdict)
    while currentday<=lastday:
        Rs=0
        Es=0
        for key in teamdict:
            teamdict[key][1]=timeupdate(c, teamdict[key][1], maxrd)
        
        ratingperioddf=premdf[(premdf['Day#'] >=currentday) & (premdf['Day#']<currentday+ratingperiodlength)].reset_index()
        for ind, row in ratingperioddf.iterrows():
            Home=row['home']
            Away=row['visitor']
            HomeRating=teamdict[Home][0]
            AwayRating=teamdict[Away][0]
            ratingdif=(HomeRating-AwayRating)/400.
            HomeLambda=np.exp(intc+ratingdif*rc+hc*h/400.)
            AwayLambda=np.exp(intc-ratingdif*rc-hc*h/400.)
            #print([Home, Away, HomeLambda, AwayLambda])
            HomeSG=np.random.poisson(HomeLambda)
            AwaySG=np.random.poisson(AwayLambda)
            teamdict[Home][8]+=1
            teamdict[Away][8]+=1
            teamdict[Home][9]+=HomeSG
            teamdict[Away][9]+=AwaySG
            teamdict[Away][10]+=HomeSG
            teamdict[Home][10]+=AwaySG
            teamdict[Home][11]+=HomeSG-AwaySG
            teamdict[Away][11]+=AwaySG-HomeSG
            if(HomeSG>AwaySG):
                teamdict[Home][4]+=1
                teamdict[Home][7]+=3
                teamdict[Away][6]+=1
            elif(HomeSG<AwaySG):
                teamdict[Away][4]+=1
                teamdict[Away][7]+=3
                teamdict[Home][6]+=1
            elif(HomeSG==AwaySG):
                teamdict[Away][5]+=1
                teamdict[Home][5]+=1
                teamdict[Home][7]+=1
                teamdict[Away][7]+=1
            HomeRd=teamdict[Home][1]
            AwayRd=teamdict[Away][1]
            HomeG=g(AwayRd**2, q)
            AwayG=g(HomeRd**2, q)
            pG=g(HomeRd**2+AwayRd**2, q)
            HomeE=E(HomeRating, AwayRating, HomeG, h, 1)
            AwayE=E(AwayRating, HomeRating, AwayG, h, -1)
            pE=E(HomeRating, AwayRating, pG, h, 1)
            HRes=Res(HomeSG, AwaySG)
            ARes=Res(AwaySG, HomeSG)
            teamdict[Home][3]+=HomeG*(HRes-HomeE)
            teamdict[Away][3]+=AwayG*(ARes-AwayE)
            teamdict[Home][2]+=insideD(HomeG, HomeE)
            teamdict[Away][2]+=insideD(AwayG, AwayE)
            Rs+=HRes
            Es+=pE
        
        h+=hk*(Rs-Es)
        for key in teamdict:
            Glickod=d2(q, teamdict[key][2])
            teamdict[key][0]=ratingupdate(teamdict[key][0],q, Glickod, teamdict[key][1], teamdict[key][3])
            teamdict[key][1]=rdupdate(teamdict[key][1], Glickod)
            teamdict[key][2]=0
            teamdict[key][3]=0
        
        currentday+=ratingperiodlength
    standings=pd.DataFrame.from_dict(teamdict, orient='index')
    standings=standings.reset_index()
    standings.columns = ['Team','Rating','Rating Deviation',
                     'Filler1','Filler2','Wins', 'Losses', 'Draws', 'Points', 'GP', 'GF', 'GA', 'GD']
    del standings['Filler1']
    del standings['Filler2']
    standings=standings.sort_values(['Points', 'GD', 'GA'], ascending=False).reset_index()
    del standings['index']
    return standings

In [ ]:
def Champions(year, nSims):
    champions=[]
    for i in range(nSims):
        standings=SeasonSimulator1st(year)
        winner=standings.iloc[0]['Team']
        if((winner!='Manchester City') & (winner!='Chelsea')&(winner!='Arsenal')&(winner!='Manchester United')&(winner!='Liverpool')&(winner!='Tottenham Hotspur')):
            print([i, winner])
        champions.append(winner)
    return champions

In [ ]:
SeasonSimulator1st(2015)

In [ ]:
YearGlickos[56]

In [ ]:
LeicesterSims=Champions(2015, 400000)

In [ ]:
def hundredpoints(nSims):
    rows=[]
    for i in range(2010, 2017):
        for j in range(nSims):
            standings=SeasonSimulator1st(2010)
            winnerpoints=standings['Points'][0]
            if(winnerpoints>99):
                #print([i, j, standings['Team'][0], winnerpoints])
                rows.append([i, standings['Team'][0], winnerpoints])
    return rows

In [63]:
LeicesterSims=Champions(2015, 400000)
teams=list(set(LeicesterSims))
rows=[]
for team in teams:
    print([team, 100*LeicesterSims.count(team)/400000])
    rows.append([team, 100*LeicesterSims.count(team)/400000])

[75, 'Swansea City']
[126, 'Everton']
[160, 'Everton']
[169, 'West Bromwich Albion']
[170, 'Everton']
[185, 'Southampton']
[245, 'Everton']
[249, 'Stoke City']
[418, 'Swansea City']
[428, 'Swansea City']
[448, 'Everton']
[482, 'Southampton']
[486, 'Sunderland']
[487, 'Everton']
[500, 'Everton']
[501, 'Everton']
[761, 'Southampton']
[822, 'Everton']
[826, 'Southampton']
[861, 'Leicester City']
[905, 'Stoke City']
[915, 'Stoke City']
[924, 'Everton']
[964, 'Swansea City']
[1045, 'Everton']
[1047, 'Swansea City']
[1077, 'Everton']
[1084, 'Everton']
[1161, 'Southampton']
[1224, 'Southampton']
[1225, 'Swansea City']
[1259, 'West Bromwich Albion']
[1325, 'Everton']
[1333, 'Southampton']
[1575, 'Stoke City']
[1579, 'Stoke City']
[1722, 'Everton']
[1727, 'Everton']
[1757, 'Leicester City']
[1758, 'Southampton']
[1795, 'Southampton']
[1804, 'Everton']
[1812, 'Everton']
[1942, 'Southampton']
[1955, 'Norwich City']
[1991, 'Stoke City']
[2131, 'Everton']
[2240, 'Stoke City']
[2266, 'Southampton']


[19978, 'Everton']
[19988, 'Southampton']
[20043, 'Southampton']
[20100, 'Everton']
[20103, 'Everton']
[20148, 'Swansea City']
[20331, 'Southampton']
[20459, 'Southampton']
[20484, 'Everton']
[20611, 'West Bromwich Albion']
[20620, 'Everton']
[20698, 'Leicester City']
[20739, 'Everton']
[20799, 'Everton']
[20837, 'Everton']
[20847, 'Everton']
[20894, 'Swansea City']
[20978, 'Everton']
[20980, 'West Bromwich Albion']
[20984, 'Everton']
[21228, 'Everton']
[21294, 'Everton']
[21297, 'Southampton']
[21303, 'Stoke City']
[21326, 'Southampton']
[21392, 'Swansea City']
[21416, 'Everton']
[21419, 'Everton']
[21444, 'West Ham United']
[21491, 'Everton']
[21507, 'Stoke City']
[21511, 'Sunderland']
[21519, 'Swansea City']
[21601, 'Everton']
[21604, 'Everton']
[21673, 'Stoke City']
[21683, 'Southampton']
[21696, 'Aston Villa']
[21800, 'Everton']
[21813, 'Stoke City']
[21865, 'Southampton']
[21911, 'Everton']
[21922, 'Southampton']
[21990, 'Stoke City']
[22149, 'Everton']
[22175, 'Everton']
[22178,

[39563, 'Southampton']
[39593, 'Everton']
[39738, 'Everton']
[39764, 'Swansea City']
[39789, 'Everton']
[39888, 'West Ham United']
[39938, 'Swansea City']
[40056, 'Everton']
[40199, 'Crystal Palace']
[40359, 'Southampton']
[40366, 'Southampton']
[40388, 'Southampton']
[40391, 'Everton']
[40393, 'Southampton']
[40464, 'Everton']
[40493, 'Everton']
[40530, 'Southampton']
[40531, 'Stoke City']
[40611, 'Everton']
[40650, 'Everton']
[40657, 'Southampton']
[40714, 'Stoke City']
[40786, 'Southampton']
[40814, 'Sunderland']
[40876, 'Southampton']
[41132, 'Swansea City']
[41172, 'Southampton']
[41213, 'Swansea City']
[41264, 'Swansea City']
[41300, 'Stoke City']
[41351, 'Everton']
[41572, 'Everton']
[41624, 'Swansea City']
[41765, 'Stoke City']
[41769, 'Southampton']
[41815, 'Southampton']
[41950, 'Everton']
[41968, 'Everton']
[41998, 'Everton']
[42042, 'Everton']
[42096, 'Stoke City']
[42153, 'Everton']
[42164, 'Newcastle United']
[42175, 'Crystal Palace']
[42227, 'Swansea City']
[42366, 'Ever

[58351, 'Everton']
[58414, 'Everton']
[58442, 'West Ham United']
[58495, 'Aston Villa']
[58644, 'Stoke City']
[58730, 'Everton']
[58762, 'Swansea City']
[58869, 'Everton']
[58947, 'Southampton']
[58987, 'Crystal Palace']
[59027, 'West Ham United']
[59038, 'West Ham United']
[59043, 'Stoke City']
[59061, 'Crystal Palace']
[59070, 'Southampton']
[59197, 'Everton']
[59246, 'Everton']
[59353, 'Stoke City']
[59380, 'West Bromwich Albion']
[59387, 'Stoke City']
[59410, 'Everton']
[59450, 'Stoke City']
[59499, 'Everton']
[59566, 'Everton']
[59598, 'Everton']
[59599, 'Swansea City']
[59651, 'Southampton']
[59694, 'Swansea City']
[59773, 'Southampton']
[59802, 'Everton']
[59877, 'Everton']
[59881, 'Swansea City']
[60003, 'Stoke City']
[60062, 'Everton']
[60131, 'Southampton']
[60192, 'Southampton']
[60202, 'Swansea City']
[60237, 'Everton']
[60275, 'Aston Villa']
[60280, 'Everton']
[60299, 'Stoke City']
[60346, 'West Ham United']
[60374, 'West Bromwich Albion']
[60378, 'Everton']
[60472, 'South

[76301, 'Southampton']
[76317, 'Southampton']
[76346, 'Everton']
[76387, 'Leicester City']
[76396, 'Everton']
[76415, 'Everton']
[76419, 'West Ham United']
[76481, 'Southampton']
[76586, 'Everton']
[76637, 'Swansea City']
[76645, 'Everton']
[76666, 'West Bromwich Albion']
[76685, 'West Ham United']
[76686, 'Everton']
[76721, 'Southampton']
[76799, 'Southampton']
[76830, 'Southampton']
[77041, 'Everton']
[77102, 'West Ham United']
[77112, 'Everton']
[77134, 'Southampton']
[77324, 'Everton']
[77331, 'Southampton']
[77451, 'Everton']
[77481, 'Stoke City']
[77604, 'Southampton']
[77632, 'Stoke City']
[77652, 'Swansea City']
[77666, 'Everton']
[77876, 'Stoke City']
[77938, 'Everton']
[77964, 'Crystal Palace']
[78036, 'Everton']
[78088, 'Stoke City']
[78099, 'Everton']
[78164, 'Southampton']
[78215, 'Everton']
[78342, 'Everton']
[78389, 'Stoke City']
[78476, 'Everton']
[78619, 'Everton']
[78824, 'Southampton']
[78873, 'Newcastle United']
[78878, 'Everton']
[78932, 'Swansea City']
[78957, 'St

...

[320030, 'Stoke City']
[320043, 'Everton']
[320054, 'Southampton']
[320059, 'Southampton']
[320134, 'Newcastle United']
[320142, 'Crystal Palace']
[320177, 'Swansea City']
[320196, 'Everton']
[320206, 'Watford']
[320420, 'Swansea City']
[320501, 'Everton']
[320598, 'Stoke City']
[320621, 'Swansea City']
[320684, 'Stoke City']
[320697, 'Stoke City']
[320779, 'Stoke City']
[320927, 'Everton']
[320992, 'Everton']
[321106, 'Everton']
[321254, 'Stoke City']
[321308, 'Everton']
[321344, 'Everton']
[321407, 'Stoke City']
[321598, 'West Bromwich Albion']
[321630, 'Everton']
[321661, 'Everton']
[321731, 'Stoke City']
[321856, 'Stoke City']
[321887, 'Stoke City']
[321899, 'Stoke City']
[321946, 'Everton']
[321959, 'West Ham United']
[322029, 'Everton']
[322061, 'Everton']
[322228, 'Swansea City']
[322393, 'Everton']
[322417, 'Everton']
[322630, 'Everton']
[322691, 'Swansea City']
[322747, 'Stoke City']
[322774, 'Southampton']
[322775, 'Everton']
[322817, 'Everton']
[322860, 'Stoke City']
[322883

[339492, 'Everton']
[339517, 'Swansea City']
[339582, 'Swansea City']
[339622, 'Southampton']
[339629, 'Everton']
[339631, 'Everton']
[339648, 'Everton']
[339677, 'Everton']
[339688, 'Everton']
[339801, 'Southampton']
[339885, 'Everton']
[339934, 'Southampton']
[339995, 'Everton']
[340024, 'Southampton']
[340027, 'Everton']
[340050, 'Swansea City']
[340051, 'Sunderland']
[340176, 'Everton']
[340209, 'Southampton']
[340216, 'Southampton']
[340279, 'Southampton']
[340343, 'Everton']
[340355, 'Everton']
[340367, 'Everton']
[340401, 'Everton']
[340523, 'Everton']
[340561, 'Stoke City']
[340570, 'Everton']
[340615, 'Everton']
[340622, 'Swansea City']
[340685, 'Everton']
[340690, 'Southampton']
[340731, 'Stoke City']
[340823, 'West Bromwich Albion']
[340825, 'Everton']
[340860, 'Everton']
[340881, 'Everton']
[340963, 'Newcastle United']
[341009, 'AFC Bournemouth']
[341085, 'Everton']
[341128, 'Stoke City']
[341349, 'Swansea City']
[341406, 'Southampton']
[341448, 'Southampton']
[341601, 'Eve

[359378, 'Everton']
[359423, 'Swansea City']
[359521, 'Stoke City']
[359618, 'Everton']
[359649, 'Stoke City']
[359679, 'Everton']
[359763, 'Everton']
[359829, 'Southampton']
[359837, 'Southampton']
[360009, 'Stoke City']
[360034, 'Southampton']
[360070, 'Southampton']
[360151, 'Southampton']
[360165, 'Everton']
[360206, 'Everton']
[360251, 'Crystal Palace']
[360446, 'Stoke City']
[360488, 'Everton']
[360603, 'Everton']
[360832, 'Everton']
[360933, 'Newcastle United']
[360936, 'Swansea City']
[361014, 'Swansea City']
[361146, 'Swansea City']
[361173, 'Everton']
[361179, 'Swansea City']
[361212, 'West Bromwich Albion']
[361216, 'Southampton']
[361280, 'Swansea City']
[361281, 'Everton']
[361295, 'Everton']
[361329, 'Southampton']
[361408, 'Everton']
[361427, 'Swansea City']
[361481, 'Everton']
[361498, 'Norwich City']
[361534, 'Southampton']
[361588, 'Stoke City']
[361622, 'Southampton']
[361720, 'Southampton']
[362002, 'Everton']
[362015, 'Stoke City']
[362037, 'Swansea City']
[362065,

[378575, 'Stoke City']
[378587, 'Stoke City']
[378640, 'Stoke City']
[378658, 'Swansea City']
[378669, 'Crystal Palace']
[378734, 'Southampton']
[378751, 'Everton']
[378835, 'Everton']
[378838, 'Stoke City']
[378966, 'Southampton']
[379144, 'Everton']
[379182, 'Swansea City']
[379184, 'Southampton']
[379198, 'Stoke City']
[379210, 'Stoke City']
[379238, 'Southampton']
[379255, 'Southampton']
[379297, 'Stoke City']
[379319, 'Swansea City']
[379380, 'Southampton']
[379583, 'Everton']
[379749, 'Southampton']
[379822, 'West Bromwich Albion']
[379842, 'Stoke City']
[379972, 'Stoke City']
[379999, 'Everton']
[380001, 'Newcastle United']
[380145, 'Everton']
[380358, 'Southampton']
[380366, 'Stoke City']
[380375, 'Stoke City']
[380389, 'Everton']
[380392, 'Swansea City']
[380412, 'Southampton']
[380438, 'West Ham United']
[380471, 'West Bromwich Albion']
[380572, 'Stoke City']
[380616, 'Southampton']
[380689, 'Swansea City']
[380719, 'Stoke City']
[380780, 'Southampton']
[380804, 'Swansea City

[396357, 'Sunderland']
[396389, 'Southampton']
[396435, 'Stoke City']
[396471, 'Everton']
[396479, 'Everton']
[396508, 'Newcastle United']
[396983, 'Southampton']
[396984, 'Southampton']
[396996, 'Norwich City']
[396998, 'Everton']
[397079, 'Everton']
[397118, 'Everton']
[397158, 'Aston Villa']
[397405, 'Stoke City']
[397408, 'Everton']
[397421, 'Southampton']
[397428, 'Everton']
[397456, 'Southampton']
[397486, 'Everton']
[397521, 'Everton']
[397542, 'Everton']
[397688, 'Swansea City']
[397748, 'Southampton']
[397804, 'Southampton']
[397823, 'Everton']
[397846, 'Swansea City']
[397860, 'Everton']
[397876, 'Everton']
[398063, 'West Bromwich Albion']
[398069, 'Stoke City']
[398126, 'Everton']
[398165, 'Everton']
[398228, 'Everton']
[398266, 'Swansea City']
[398304, 'Stoke City']
[398363, 'Swansea City']
[398415, 'Southampton']
[398443, 'West Ham United']
[398444, 'Swansea City']
[398479, 'Aston Villa']
[398495, 'Everton']
[398531, 'Everton']
[398544, 'Swansea City']
[398621, 'Southampto

['Crystal Palace', 0.041]
['Tottenham Hotspur', 3.25925]
['Watford', 0.0075]
['AFC Bournemouth', 0.00475]
['Swansea City', 0.1915]
['Manchester City', 26.8695]
['Norwich City', 0.02075]
['West Bromwich Albion', 0.0415]
['West Ham United', 0.05075]
['Aston Villa', 0.013]
['Newcastle United', 0.03]
['Sunderland', 0.02525]
['Manchester United', 9.23575]
['Arsenal', 15.0535]
['Stoke City', 0.2435]
['Liverpool', 4.109]
['Chelsea', 39.547]
['Southampton', 0.44325]
['Leicester City', 0.036]
['Everton', 0.77725]


In [65]:
columns=['Team', 'Probability']
GlickoProbs=pd.DataFrame(rows,columns=columns)
GlickoProbs.sort_values('Probability', ascending=False)

,Team,Probability
16,Chelsea,39.54700
5,Manchester City,26.86950
13,Arsenal,15.05350
12,Manchester United,9.23575
15,Liverpool,4.10900
1,Tottenham Hotspur,3.25925
19,Everton,0.77725
17,Southampton,0.44325
14,Stoke City,0.24350
4,Swansea City,0.19150


In [ ]:
SeasonSimulator1st(2015)

In [ ]:
Prem18=pd.read_csv('201718Prem.csv')

def gettime(str):
    split=str.split('/')
    first='20'+split[2]
    if len(split[1])==1:
        split[1]='0'+split[1]
    str2=first+'-'+split[1]+'-'+split[0]
    #datetime.strptime(D, '%Y-%m-%d')
    return datetime.strptime(str2, '%Y-%m-%d')
Prem18['Date']=Prem18.apply(lambda row: gettime(row['Date']), axis=1)
Prem18

In [ ]:
#Prem18['Date']=Prem18.apply(lambda row: gettime(row['Date']), axis=1)


#def DT(Date):
 #   return datetime.strptime(Date, '%Y-%m-%d')
#Prem18['Date']=Prem18.apply(lambda row: DT(row['Date']), axis=1)

def to_integer(current_time):
    return (current_time-fulldf['Date'][0]).days
Prem18['Day#']=Prem18.apply(lambda row: to_integer(row['Date']), axis=1)

In [ ]:
Prem18

In [54]:
best2

{'b': 2197.379482208093,
 'c': 0.0003054360773678544,
 'hk': 0.2752882185619584,
 'maxrd': 77.77234622658803}

In [ ]:
import copy
for i in range(len(fulldf['home'].unique())):
    Team=fulldf['home'].unique()[i]
    Teamdf=fulldf[fulldf['home']==Team].reset_index()
    Tier=int(Teamdf['tier'][0])
    LastMatch=fulldf['Day#'][0]
    dcalc=0
    scalc=0
    initialrow=[Team,initvalues[Tier-1], Rd,Tier,  dcalc,scalc ]
    initialrows.append(initialrow)
#Needs to be rewritten to optimize using BuildEloDict Function
for a in range(len(initialrows)):
    initdictionary[initialrows[a][0]]=initialrows[a][1:]

def SeasonBoost(rd, b, maxrd):
    return min(maxrd, np.sqrt(rd**2+b))

def PromotionBoost(rd, p, maxrd):
    return min(maxrd, np.sqrt(rd**2+p))

def Glickologlikboost(params):
    dicts=[]
    rows2=[]
    rows3=[]
    ratingperiodlength=7.0
    c=params['c']
    h=80
    hk=params['hk']
    b=params['b']
    #minrd=params['minrd']
    maxrd=params['maxrd']
    curtime=time.time()
    rows=len(fulldf)
    q=np.log(10)/400
    logliks=0.0
    hburn=10
    burn=25
    startyear=fulldf['Season'].values[0]
    #print(startyear)
    year=copy.deepcopy(startyear)
    startday=fulldf['Season']
    Glickodictionary=copy.deepcopy(initdictionary)
    for key in Glickodictionary:
        Glickodictionary[key][1]=maxrd
    currentyear=fulldf['Season'].values[0]
    startdate=fulldf['Date'][0]
    GameCounter=0.
    Rs=0.
    Es=0.
    totaltime=fulldf['Day#'][rows-1]-fulldf['Day#'][0]
    day=0.
    #print(Glickodictionary)
    while day<=totaltime:
        for key in Glickodictionary:
            Glickodictionary[key][1]=timeupdate(c, Glickodictionary[key][1], maxrd)
        ratingperioddf=fulldf[(fulldf['Day#'] >=day) & (fulldf['Day#']<day+ratingperiodlength)].reset_index()
        if len(ratingperioddf!=0):
            Rs=0.
            Es=0.

            for ind, row in ratingperioddf.iterrows():
                if(row['Season']!=year):
                    #print(startyear)
                    if(row['Season']>startyear+hburn):
                        newdf2=fulldf[fulldf['Season']==row['Season']]
                        for key in Glickodictionary:
                            teamdf=newdf2[newdf2['home']==key].reset_index()
                            if(len(teamdf)!=0):
                                firstgame=teamdf.iloc[0]
                                Glickodictionary[key][2]=firstgame['tier']
                                Glickodictionary[key][1]=SeasonBoost(Glickodictionary[key][1], b, maxrd)
                    dicts.append([row['Season'],h,  copy.deepcopy(Glickodictionary)])
                         
                #print('sup')
                Home=row['home']
                Away=row['visitor']
                HomeGoals=row['hgoal']
                AwayGoals=row['vgoal']
                year=row['Season']
                date=row['Date']
                tier=row['tier']
                Homestats=Glickodictionary[Home]
                Awaystats=Glickodictionary[Away]
                HomeRating=Homestats[0]
                AwayRating=Awaystats[0]
                HomeRd=Homestats[1]
                AwayRd=Awaystats[1]
                HomeG=g(AwayRd**2, q)
                AwayG=g(HomeRd**2, q)
                pG=g(HomeRd**2+AwayRd**2, q)
                HomeE=E(HomeRating, AwayRating, HomeG, h, 1)
                AwayE=E(AwayRating, HomeRating, AwayG, h, -1)
                pE=E(HomeRating, AwayRating, pG, h, 1)
                HRes=Res(HomeGoals, AwayGoals)
                ARes=Res(AwayGoals, HomeGoals)

                Glickodictionary[Home][4]+=HomeG*(HRes-HomeE)
                Glickodictionary[Away][4]+=AwayG*(ARes-AwayE)
                Glickodictionary[Home][3]+=insideD(HomeG, HomeE)
                Glickodictionary[Away][3]+=insideD(AwayG, AwayE)
                Rs+=HRes
                Es+=pE
                row=[Home, Away,date, year,tier, HomeRating, AwayRating, h, HomeRd, AwayRd, pG, HomeGoals, AwayGoals, HRes, pE]
                rows2.append(row)
                if(year>startyear+burn):
                    logliks+=lik(pE, HRes)
                    

            if(year>startyear+hburn):
                h+=hk*(Rs-Es)

            for key in Glickodictionary:
                Glickod=d2(q, Glickodictionary[key][3])
                Glickodictionary[key][0]=ratingupdate(Glickodictionary[key][0],q, Glickod, Glickodictionary[key][1], Glickodictionary[key][4])
                Glickodictionary[key][1]=rdupdate(Glickodictionary[key][1], Glickod)
                Glickodictionary[key][3]=0
                Glickodictionary[key][4]=0
        rows3.append([date, Glickodictionary['Tottenham Hotspur'][1]])  
        day+=ratingperiodlength
    print(params ,logliks,Glickodictionary['Tottenham Hotspur'][0], time.time()-curtime)
    #print(Glickodictionary)
    return dicts

In [ ]:
best2={'b': 2197.379482208093,
 'c': 0.0003054360773678544,
 'hk': 0.2752882185619584,
 'maxrd': 77.77234622658803}
Glickologlikboost(best2)